This cookie cutter notebook is intended to help guide the process from a city-specific rain gage csv format, to a generic NetCDF containing timeSeries data and with attributes explaining how the file was created. For best results, make a copy of this notebook and delete any cells that you don't need. 

# Roadmap
 1. [Inspect file](#1.-Inspect-file)
 2. [Read data into pandas.Dataframe object](#2.-Read-data-into-pandas.Dataframe-object)
 3. [Read locations into pandas.Dataframe object](#3.-Read-locations-into-pandas.Dataframe-object)
 4. [Figure out how these two Dataframes overlap](#4.-Figure-out-how-these-two-Dataframes-overlap)
 5. [Write a function that reads all the data files](#5.-Write-a-function-that-reads-all-the-data-files)
 6. [QAQC](#6.-QAQC)
     - [missing loc](#missing-loc) 
     - [TZ check](#TZ-check)
     - [bad values](#bad-values)
     - [units check](#units-check)
 7. [Write to NetCDF](#7.-Write-to-NetCDF)

## 1. Inspect file
If there is going to be something strange in the data you can bet it'll be in the beginning or end. So we will look at those first to try to spot stupid stuff

In [ ]:
!head -4 /home/jsignell/data/BALTIMORE/BaltoCounty/8-31-2015.xlsx.csv

In [ ]:
!tail -4 /home/jsignell/data/BALTIMORE/BaltoCounty/8-31-2015.xlsx.csv

## 2. Read data into pandas.Dataframe object
This is probably the most iterative step. By the end you want a time parsed and indexed dataframe

In [ ]:
import os
import pandas as pd
import xarray as xr

In [ ]:
f = '/home/jsignell/data/BALTIMORE/BaltoCounty/8-31-2015.xlsx.csv'

In [ ]:
gage = pd.read_csv(f, parse_dates=['Time'], index_col='Time', skipfooter=1, engine='python')
gage.index.name = 'time'
gage.head()

## 3. Read locations into pandas.Dataframe object
The locations can be wonky, but the files are small and locations matter a lot, so don't skimp on the data wrangling at this stage. 

In [ ]:
f = '/home/jsignell/data/BALTIMORE/Location_names.txt'

In [ ]:
locs = pd.read_csv(f, delim_whitespace=True, header=None,
                   skiprows=45, skipfooter=1, usecols=[2,3,4],
                   engine='python', index_col=0)
locs.index.name='station'
locs.columns = ['lon', 'lat']
locs.head()

After aquiring this file, we were given another file with some different stations and some overlap. Let's see how different the files really are. 

In [ ]:
f1 = '/home/jsignell/data/BALTIMORE/County_BES_rain_gages.xlsx'

In [ ]:
locs1 = pd.read_excel(f1, index_col=0)
locs1.head()

So we want to know if there are gages in locs1 that aren't in locs, and we want to know if the gages that are in both are in the same locations. 

In [ ]:
set(locs1.index) - set(locs.index)

In [ ]:
compare = locs1.join(locs, how='inner')
compare[(compare.Latitude != compare.lat.round(4)) | (compare.Longitude != compare.lon.round(4))]

It looks like for our purposes locs includes more precise location data, so that is probably the better choice.

## 4. Figure out how these two Dataframes overlap
We will first look at the names of the gages in both dataframes, and then see what kind of parsing is going to be needed to get matching gage names. In the case below, the units (in) are included in the variable names in the gage data, but not in the locations. So we will need to strip this part of each string off.

In [ ]:
gage.columns

In [ ]:
locs.index

In [ ]:
col = gage.columns[0]
col.split('-')[0]

In [ ]:
columns = [col.split('-')[0].split('.')[0] for col in gage.columns]

We don't have the locations for these gages

In [ ]:
set(columns) - set(locs.index)

But we can still keep moving forward. 

In [ ]:
gage.columns = columns

In [ ]:
gage.head()

## 5. Write a function that reads all the data files
Now that we have a sense of what we are looking at and how the gage files and the location files overlap, we can write a function that we will proceeed to use on each of the files. Use whatever you learned above about the structure of the files. 

In [ ]:
path = '/home/jsignell/data/BALTIMORE/BaltoCounty/'

In [ ]:
files = os.listdir(path)

In [ ]:
def func(f):
    # read file in using the default style
    gage = pd.read_csv(f, parse_dates=['Time'], index_col='Time')
    
    # check that date time has been parsed and if it hasn't try to fix it
    if gage.index.dtype != '<M8[ns]':
        gage = pd.read_csv(f, parse_dates=['Time'], index_col='Time',
                           skipfooter=1, engine='python')
    
    # name index time
    gage.index.name = 'time'
    
    # set gage_names to match those in the locs dataframe
    #gage.columns = [col.split('-')[0].split('.')[0] for col in gage.columns]
    columns = []
    for col in gage.columns:
        col = col.split('-')[0].split('.')[0]
        if col.endswith('RG'):
            col = col[:-2]
        columns.append(col)
    gage.columns = columns

    return gage

In [ ]:
%%time
df_list = []
for f in files:
    df_list.append(func(path+f))

In [ ]:
df = pd.concat(df_list)
df = df.sort_index()

df.columns.name = 'station'

## 6. QAQC
[missing loc](#missing-loc) | [TZ check](#TZ-check) | [bad values](#bad-values)

### bad values
This is probably the most decision based QAQC check, but sometimes it is very easy to toss out values because they are ridiculous. This is the case when there are values that exceed the upper extents of the possible. We won't try to figure out what these values should be. We will just set them to NAN.

In [ ]:
df.max().sort_values(ascending=False)[:10]

In [ ]:
%matplotlib notebook
plt.plot(df['HR18'])

In [ ]:
%matplotlib notebook
plt.plot(df['GF10'])

In [ ]:
%matplotlib notebook
plt.plot(df['BC53'])

In [ ]:
df.loc['2009-08-10':'2009-08-29','HR18'] = pd.np.nan
df.loc['2013-05-14':'2013-05-18','GF10'] = pd.np.nan
df.loc['2016-05-17':'2016-05-18','BC53'] = pd.np.nan

### missing loc
We can visually represent data availability, and see which gages aren't listed in our locs dataframe

In [ ]:
set(df.columns) - set(locs.index)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.figure(figsize=(8,8))
plt.pcolormesh(df.isnull().values)

#### Note:
There are almost always the same number of gages, but their locations or possibly just their names change at a certaion point in time. We can't resolve this by being clever or guessing. We need to go back to the source and figure it out. Or we need to provide masked locations for the gages that don't have locations listed. That way if we figure things out at a later date, we can just add that info in rather than coming back to the beginning.

#### External Input:
We were told that gages that end in A are at the same locations as their A-less peers

In [ ]:
def combine_columns(df, cols):
    '''Combine given columns in a dataframe and retain name from first column'''
    bar = pd.concat([df[col] for col in cols])
    df[cols[0]] = bar.dropna().sort_index().drop_duplicates()
    return df.drop(cols[1:], axis=1)

In [ ]:
df = combine_columns(df, ['HR16', 'HR16/HR16A', 'HR16A'])
df = combine_columns(df, ['BC40', 'BC40A'])
df = combine_columns(df, ['BC43', 'BC43A'])
df = combine_columns(df, ['BC50', 'BC50A'])

In [ ]:
set(df.columns) - set(locs.index)

### TZ check
Time zone is very important and not often carried around in the metadata. If it is, that is awesome, and we can use it now. It is always best to ask around and see if you can figure out what you have. Guessing timezones bases on data is a tricky and time consuming business. 

If we don't know the time zone - as is often the case - we can try to find a storm and use radar data to make a best guess. Normally there are three viable options for timezones: UTC, local standard time, and local time. By this I mean that people can either take or leave the daylight savings part. The storm should be in what would be daylight savings time (winter) so that we can be sure to test whether daylight savings is used or not.

In [ ]:
df.mean(axis=1).plot()

Now in order to get a sense of this storm we will compare the rain gage data from the storm with the radar data. So we can save the storm off into a csv, and then load it in the TZ checking notebook running in in the radar environment. This allows us to download all the radar data from the storm, calculate rainfall, pull out rainfall at the gage locations, and then resample and take the mean to get the average 5min rain rate over the gages. 

In [ ]:
locs.join(df['2015-11-19 10:00':'2015-11-19 20:00'].T, how='inner').to_csv('storm_gage.csv')

In [ ]:
%%HTML
<img src=tmp/UTC_2015-11-19.png/ width=400/>
<img src=tmp/US_Eastern_2015-11-19.png/ width=400/>
<img src=tmp/EST_2015-11-19.png/ width=400/>

If you need to check your timezone naming options. 

In [ ]:
from pytz import all_timezones
from pytz import common_timezones

Looks like EST

In [ ]:
# set timezone and convert to UTC
df.index = df.index.tz_localize('EST').tz_convert('UTC')

### units check
We just know it is in inches, so let's convert to mm

In [ ]:
df*=25.4

## 7. Write to NetCDF

In [ ]:
DATA_PATH = '/home/jsignell/erddapData/UrbanRainfall/Baltimore/'
RGN='Baltimore_County'

In [ ]:
site = 'Baltimore County'

units = 'mm'
tz = 'storm_guessed'
calc_from_tips = False
label = 'right'
freq = '5min'
per_hour = 12

In [ ]:
df.index = df.index.astype('datetime64[ns]')

datasets = [xr.DataArray(df[col]) for col in df.columns]
combined = xr.concat(datasets, 'station')
ds0 = combined.to_dataset(name='rain_gage')
ds0['rain_gage'].attrs.update({'units': units, 'standard_name': 'gage rain depth',
                               'label': label, 'freq': freq, 'per_hour': per_hour,
                               'tz': tz, 'calc_from_tips': calc_from_tips})
#ds0['rain_gage'].encoding.update({'chunksizes': (5, 10000), 'zlib': True})
ds0['station'] = df.columns

In [ ]:
# we only care about locations where we have gage data
s = df.mean(axis=0)
s.name = 'historical_mean'
locs = locs.join(s, how='inner')[['lon', 'lat']]

ds1 = xr.Dataset.from_dataframe(locs)

In [ ]:
ds_ = ds0.merge(ds1)
ds_.set_coords(['lon', 'lat'], inplace=True)
ds_['station'] = ds_['station'].astype(str)
ds_

In [ ]:
ds_.station.attrs.update({'standard_name': 'station_name', 'long_name': 'station name', 'cf_role':'timeseries_id'})
ds_.lat.attrs.update({'standard_name': 'latitude', 'long_name':'station latitude', 'units': 'degrees_north'})
ds_.lon.attrs.update({'standard_name': 'longitude', 'long_name':'station longitude', 'units': 'degrees_east'})
ds_.time.encoding = {'units':'seconds since 1970-01-01', 'calendar':'gregorian', 'dtype': pd.np.double}
#                     'chunksizes': (10000,), 'zlib': True}

ds_.attrs.update({'description': '{site} rain gage network'.format(site=site),
                  'history': 'Created {now}'.format(now=pd.datetime.now()),
                  'Conventions': "CF-1.6",
                  'featureType': 'timeSeries'})

In [ ]:
for year in range(pd.Timestamp(ds_.time[0].values).year,
                  pd.Timestamp(ds_.time[-1].values).year+1):
    if not os.path.isdir(DATA_PATH + str(year)):
        os.mkdir(DATA_PATH + str(year))
    ds_.sel(time=str(year)).to_netcdf('{path}{year}/{RGN}_freq.nc'.format(
            path=DATA_PATH, year=year, RGN=RGN), format='netCDF4', engine='h5netcdf')